In [1]:
from google.colab import drive
drive.mount('/content/drive/')
import shutil
from google.colab import files

Mounted at /content/drive/


In [2]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras.layers import Dense, Flatten, LSTM, Dropout, SimpleRNN, BatchNormalization
from keras.models import Sequential
from sklearn.metrics import confusion_matrix, accuracy_score
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler
from keras.utils import to_categorical

In [3]:

data_path = "/content/drive/MyDrive/Location_Season_Data/"

In [4]:
df = pd.read_csv(data_path + "Location_Season_Pressure_first_3_year_ROI.csv")
df

,0,1,2,3,4,5,6,7,8,9,...,223,224,225,226,227,228,season,Name,Day,Class
0,0.475692,0.473846,0.471385,0.468923,0.465846,0.462769,0.460308,0.456615,0.453538,0.450462,...,0.435077,0.443077,1,1,1,0,2,116_c7_cp2_100_32,116,0
1,0.464615,0.463385,0.462769,0.460923,0.460308,0.457846,0.455385,0.452923,0.451077,0.450462,...,0.454769,0.475077,1,0,1,0,2,125_c3_cp2_49_32,125,0
2,0.248615,0.243692,0.240000,0.237538,0.235692,0.234462,0.234462,0.235692,0.237538,0.240615,...,0.246154,0.264615,1,0,0,0,2,120_c1_cp2_31_29,120,0
3,0.552000,0.544000,0.536615,0.529846,0.524923,0.521846,0.520000,0.519385,0.520615,0.523077,...,0.475077,0.492923,1,1,1,0,2,127_c3_cp3_96_38,127,0
4,0.248615,0.243692,0.240000,0.237538,0.235692,0.234462,0.234462,0.235692,0.237538,0.240615,...,0.246154,0.264615,1,0,0,0,2,120_c2_cp2_31_29,120,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1063,0.930833,0.935248,0.939601,0.937135,0.942242,0.940599,0.940394,0.935167,0.934217,0.936547,...,0.948077,0.953185,1,0,1,1,3,997_c12_cp2_62_59,997,3
1064,0.869459,0.873418,0.876279,0.877193,0.879589,0.879725,0.879310,0.874263,0.870889,0.869159,...,0.911058,0.919884,1,0,1,1,3,998_c12_cp2_37_48,998,2
1065,0.759864,0.772639,0.783731,0.791910,0.800783,0.806087,0.810837,0.810413,0.810015,0.810133,...,0.813462,0.813224,1,1,1,0,3,999_c11_cp2_102_34,999,2
1066,0.812957,0.820837,0.826595,0.829435,0.835047,0.838979,0.843350,0.843320,0.844379,0.848008,...,0.909615,0.921332,1,0,1,0,3,999_c11_cp3_37_44,999,2


In [ ]:
X = df.drop(columns = ["225", "226", "227", "228", "season", "Name", "Day", "Class"])
Y = df["Class"]

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 42)

In [ ]:
#Reshape training and testing data
train_X = X_train.to_numpy().reshape(X_train.shape[0],15,15,1).astype("float32")

train_Y = Y_train.to_numpy()

test_X = X_test.to_numpy().reshape(X_test.shape[0],15,15,1).astype("float32")

test_Y = Y_test.to_numpy()

train_Y = to_categorical(train_Y, num_classes=4)
test_Y = to_categorical(test_Y, num_classes=4)

In [ ]:
#Define the model architecture
model_rnn = Sequential()
model_rnn.add(LSTM(8, input_shape=(15, 15), return_sequences=True))
model_rnn.add(BatchNormalization())
model_rnn.add(Dropout(0.2))

model_rnn.add(LSTM(8, return_sequences=True))
model_rnn.add(BatchNormalization())
model_rnn.add(Dropout(0.2))

model_rnn.add(LSTM(8))
model_rnn.add(BatchNormalization())

model_rnn.add(Dense(4, activation='softmax'))


In [ ]:
model_rnn.compile(loss = "categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])

In [ ]:
early_stopping = EarlyStopping(monitor='loss', patience=30)


#Fit the model
model_rnn.fit(train_X, train_Y, epochs = 1000, batch_size = 32, verbose = 2, callbacks = early_stopping)

Epoch 1/1000
27/27 - 9s - loss: 1.3225 - accuracy: 0.3770 - 9s/epoch - 321ms/step
Epoch 2/1000
27/27 - 1s - loss: 1.1675 - accuracy: 0.5012 - 639ms/epoch - 24ms/step
Epoch 3/1000
27/27 - 1s - loss: 1.0591 - accuracy: 0.5105 - 570ms/epoch - 21ms/step
Epoch 4/1000
27/27 - 1s - loss: 1.0223 - accuracy: 0.5211 - 576ms/epoch - 21ms/step
Epoch 5/1000
27/27 - 1s - loss: 1.0228 - accuracy: 0.5363 - 554ms/epoch - 21ms/step
Epoch 6/1000
27/27 - 1s - loss: 0.9708 - accuracy: 0.5808 - 544ms/epoch - 20ms/step
Epoch 7/1000
27/27 - 1s - loss: 0.9763 - accuracy: 0.5632 - 529ms/epoch - 20ms/step
Epoch 8/1000
27/27 - 1s - loss: 0.9497 - accuracy: 0.5738 - 705ms/epoch - 26ms/step
Epoch 9/1000
27/27 - 1s - loss: 0.9516 - accuracy: 0.5656 - 654ms/epoch - 24ms/step
Epoch 10/1000
27/27 - 1s - loss: 0.9531 - accuracy: 0.5632 - 836ms/epoch - 31ms/step
Epoch 11/1000
27/27 - 1s - loss: 0.9294 - accuracy: 0.5808 - 815ms/epoch - 30ms/step
Epoch 12/1000
27/27 - 1s - loss: 0.9296 - accuracy: 0.5890 - 822ms/epoch - 3

In [ ]:
# predict class probabilities for test set
Y_pred = model_rnn.predict(test_X)

# convert predicted probabilities to class labels
Y_pred_labels = np.argmax(Y_pred, axis=1)


# convert true labels to class labels
Y_true_labels = np.argmax(test_Y, axis=1)

#Accuracy
accuracy = accuracy_score(Y_true_labels, Y_pred_labels)
print(accuracy)

# compute confusion matrix
conf_matrix = confusion_matrix(Y_true_labels, Y_pred_labels)

print(conf_matrix)

7/7 [==============================] - 1s 6ms/step
0.8598130841121495
[[42  0 13  0]
 [ 0 48  0  1]
 [ 5  0 71  6]
 [ 0  1  4 23]]


In [5]:
df_train = df[(df["Day"] >= 0) & (df["Day"] <= 730)]
df_train

,0,1,2,3,4,5,6,7,8,9,...,223,224,225,226,227,228,season,Name,Day,Class
0,0.475692,0.473846,0.471385,0.468923,0.465846,0.462769,0.460308,0.456615,0.453538,0.450462,...,0.435077,0.443077,1,1,1,0,2,116_c7_cp2_100_32,116,0
1,0.464615,0.463385,0.462769,0.460923,0.460308,0.457846,0.455385,0.452923,0.451077,0.450462,...,0.454769,0.475077,1,0,1,0,2,125_c3_cp2_49_32,125,0
2,0.248615,0.243692,0.240000,0.237538,0.235692,0.234462,0.234462,0.235692,0.237538,0.240615,...,0.246154,0.264615,1,0,0,0,2,120_c1_cp2_31_29,120,0
3,0.552000,0.544000,0.536615,0.529846,0.524923,0.521846,0.520000,0.519385,0.520615,0.523077,...,0.475077,0.492923,1,1,1,0,2,127_c3_cp3_96_38,127,0
4,0.248615,0.243692,0.240000,0.237538,0.235692,0.234462,0.234462,0.235692,0.237538,0.240615,...,0.246154,0.264615,1,0,0,0,2,120_c2_cp2_31_29,120,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
733,0.912615,0.915692,0.923692,0.929846,0.932308,0.935385,0.936615,0.937231,0.937231,0.936615,...,0.927385,0.928615,1,1,1,1,4,643_c12_cp2_96_62,643,3
734,0.946462,0.944615,0.944615,0.942154,0.938462,0.932308,0.929231,0.933538,0.936615,0.936000,...,0.936000,0.939077,1,1,1,1,3,620_c11_cp2_86_60,620,3
735,0.856000,0.864000,0.874462,0.876923,0.873846,0.864000,0.868923,0.872615,0.880615,0.889846,...,0.931077,0.927385,1,1,1,1,4,694_c17_cp2_86_63,694,3
736,0.935385,0.940923,0.942769,0.942769,0.942769,0.945231,0.945846,0.944000,0.942154,0.938462,...,0.944000,0.946462,1,0,1,1,3,616_c10_cp2_67_57,616,3


In [6]:
df_test = df[(df["Day"] >= 731) & (df["Day"] <= 1095)]
df_test

,0,1,2,3,4,5,6,7,8,9,...,223,224,225,226,227,228,season,Name,Day,Class
332,0.462154,0.472000,0.480615,0.488615,0.494154,0.497846,0.502154,0.504000,0.504000,0.504000,...,0.534154,0.527385,1,0,0,0,1,731_c12_cp1_24_27,731,1
737,0.864588,0.866602,0.869459,0.871832,0.876163,0.879234,0.880788,0.876719,0.873343,0.870635,...,0.900000,0.903958,1,0,1,1,4,1001_c12_cp2_63_48,1001,3
738,0.894788,0.890944,0.886508,0.881579,0.878610,0.873343,0.867980,0.857073,0.847324,0.840630,...,0.871635,0.880309,1,0,1,0,4,1002_c11_cp2_41_44,1002,2
739,0.870434,0.883155,0.895762,0.906920,0.919726,0.929799,0.939409,0.941552,0.944035,0.946877,...,0.926923,0.923263,1,0,0,1,4,1002_c15_cp1_20_48,1002,1
740,0.934730,0.930867,0.926936,0.922027,0.920215,0.916053,0.912315,0.903242,0.896416,0.892277,...,0.917308,0.920849,1,0,1,1,4,1003_c11_cp2_38_48,1003,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1063,0.930833,0.935248,0.939601,0.937135,0.942242,0.940599,0.940394,0.935167,0.934217,0.936547,...,0.948077,0.953185,1,0,1,1,3,997_c12_cp2_62_59,997,3
1064,0.869459,0.873418,0.876279,0.877193,0.879589,0.879725,0.879310,0.874263,0.870889,0.869159,...,0.911058,0.919884,1,0,1,1,3,998_c12_cp2_37_48,998,2
1065,0.759864,0.772639,0.783731,0.791910,0.800783,0.806087,0.810837,0.810413,0.810015,0.810133,...,0.813462,0.813224,1,1,1,0,3,999_c11_cp2_102_34,999,2
1066,0.812957,0.820837,0.826595,0.829435,0.835047,0.838979,0.843350,0.843320,0.844379,0.848008,...,0.909615,0.921332,1,0,1,0,3,999_c11_cp3_37_44,999,2


In [7]:
labels_train = df_train["Class"]
labels_test = df_test["Class"]

data_train = df_train.drop(columns = ["225", "226", "227", "228", "season", "Name", "Day", "Class"])
data_test = df_test.drop(columns = ["225", "226", "227", "228", "season", "Name", "Day", "Class"])

In [8]:
#Reshape training and testing data
data_train = data_train.to_numpy().reshape(data_train.shape[0],15,15,1).astype("float32")

labels_train = labels_train.to_numpy()

data_test = data_test.to_numpy().reshape(data_test.shape[0],15,15,1).astype("float32")

labels_test = labels_test.to_numpy()

labels_train = to_categorical(labels_train, num_classes=4)
labels_test = to_categorical(labels_test, num_classes=4)

In [89]:
from keras.layers import Input, LSTM, Dropout, Dense, BatchNormalization, Layer, Lambda
from keras.models import Model
import keras.backend as K

# Custom Attention Layer
class AttentionLayer(Layer):
    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.W_q = self.add_weight(name='W_q',
                                   shape=(input_shape[-1], input_shape[-1]),
                                   initializer='uniform',
                                   trainable=True)
        self.W_v = self.add_weight(name='W_v',
                                   shape=(input_shape[-1], input_shape[-1]),
                                   initializer='uniform',
                                   trainable=True)
        super(AttentionLayer, self).build(input_shape)

    def call(self, x):
        q = K.dot(x, self.W_q)
        v = K.dot(x, self.W_v)

        scores = K.batch_dot(q, K.permute_dimensions(v, (0, 2, 1)))
        attention_weights = K.softmax(scores)

        return K.batch_dot(attention_weights, x)

    def compute_output_shape(self, input_shape):
        return input_shape


# Reshape the data to (batch_size, sequence_length, feature_dim)
data_train = data_train.reshape(data_train.shape[0], 15, 15).astype("float32")
data_test = data_test.reshape(data_test.shape[0], 15, 15).astype("float32")

# Define the LSTM model with Attention
input_layer = Input(shape=(15, 15))
lstm = LSTM(64, return_sequences=True)(input_layer)
batch_norm = BatchNormalization()(lstm)
dropout = Dropout(0.1)(batch_norm)

# Add Attention Layer
attention = AttentionLayer()(dropout)

# Extract the last time step of the attention output for classification
last_time_step = Lambda(lambda x: x[:, -1, :])(attention)

# Final Dense Layer for classification
output_layer = Dense(4, activation='softmax')(last_time_step)

# Create the model
model_attention = Model(inputs=input_layer, outputs=output_layer)

# Compile the model
model_attention.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Display the model summary
model_attention.summary()

Model: "model_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_9 (InputLayer)        [(None, 15, 15)]          0         
                                                                 
 lstm_8 (LSTM)               (None, 15, 64)            20480     
                                                                 
 batch_normalization_8 (Batc  (None, 15, 64)           256       
 hNormalization)                                                 
                                                                 
 dropout_8 (Dropout)         (None, 15, 64)            0         
                                                                 
 attention_layer_8 (Attentio  (None, 15, 64)           8192      
 nLayer)                                                         
                                                                 
 lambda_8 (Lambda)           (None, 64)                0   

In [91]:
# Fit the model
early_stopping = EarlyStopping(monitor='loss', patience=30)

#Fit the model
model_attention.fit(data_train, labels_train, epochs = 400, batch_size = 64, verbose = 1)

Epoch 1/400
12/12 [==============================] - 5s 39ms/step - loss: 0.9825 - accuracy: 0.5550
Epoch 2/400
12/12 [==============================] - 1s 41ms/step - loss: 0.8112 - accuracy: 0.6377
Epoch 3/400
12/12 [==============================] - 0s 41ms/step - loss: 0.7378 - accuracy: 0.6879
Epoch 4/400
12/12 [==============================] - 0s 39ms/step - loss: 0.6344 - accuracy: 0.7246
Epoch 5/400
12/12 [==============================] - 0s 40ms/step - loss: 0.5938 - accuracy: 0.7544
Epoch 6/400
12/12 [==============================] - 0s 29ms/step - loss: 0.5621 - accuracy: 0.7707
Epoch 7/400
12/12 [==============================] - 0s 27ms/step - loss: 0.5700 - accuracy: 0.7612
Epoch 8/400
12/12 [==============================] - 0s 35ms/step - loss: 0.5675 - accuracy: 0.7571
Epoch 9/400
12/12 [==============================] - 0s 41ms/step - loss: 0.5286 - accuracy: 0.7734
Epoch 10/400
12/12 [==============================] - 1s 47ms/step - loss: 0.5020 - accuracy: 0.7856

In [92]:
# predict class probabilities for test set
Y_pred1 = model_attention.predict(data_test)

# convert predicted probabilities to class labels
Y_pred_labels1 = np.argmax(Y_pred1, axis=1)


# convert true labels to class labels
Y_true_labels1 = np.argmax(labels_test, axis=1)

#Accuracy
accuracy1 = accuracy_score(Y_true_labels1, Y_pred_labels1)
print(accuracy1)

# compute confusion matrix
conf_matrix1 = confusion_matrix(Y_true_labels1, Y_pred_labels1)

print(conf_matrix1)

11/11 [==============================] - 1s 8ms/step
0.7583081570996979
[[76 11  2  0]
 [ 1 77  1  0]
 [34  2 48  3]
 [ 3 12 11 50]]
